<a href="https://colab.research.google.com/github/Ksenia-90/Machine_learning-_in_business/blob/lesson_8/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Занятие 8. АБ-тестирование (проверяем все в онлайн-экспериментах)##

#В ПРОЦЕССЕ ДОРАБОТКИ

*Задание *

Провести аналогичные исследования но для чека покупок клиентов (колонки revenue)
в интернет магазине есть 2-ва типа продуктов:

100 руб.
250 руб.
ну и конечно клиент может ничего не купить
провести исследование изменился ли средний чек

In [204]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
import pandas as pd
import numpy as np
import random
import scipy
import scipy as sp
import scipy.stats as sts
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(42)

In [206]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ab_datas.csv")
df.head()

,user_id,timestamp,group,landing_page,converted,revenue
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,100
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,250


In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
 5   revenue       294478 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 13.5+ MB


In [208]:
#df = df.drop(np.where(df['revenue'] == 0)[0])
#df['revenue'] = df['revenue'].replace({100: 1, 250: 1})

In [209]:
n_rows = df.shape[0]
n_rows

294478

In [210]:
user_total = df['user_id'].nunique()
user_total

290584

In [211]:
revenue_total = df['revenue'].nunique()
revenue_total

3

In [212]:
df[df.revenue == 0].shape[0]/df.shape[0]

0.41768824835811164

In [213]:
df[df.revenue == 1].shape[0]/df.shape[0]

0.0

In [214]:
df[((df.group=='treatment') & (df.landing_page!='new_page')) | 
   ((df.group!='treatment') & (df.landing_page=='new_page'))].shape[0]

3893

In [215]:
df2 = df.drop(df[((df['group'] == 'treatment') ^ (df['landing_page'] == 'new_page'))].index, axis=0)
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]


0

In [216]:
df.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
revenue         0
dtype: int64

In [217]:
ctrl = df2[df2.group=='control']
p_ctrl = ctrl.converted.sum()/ctrl.shape[0]
p_ctrl

0.1203863045004612

In [218]:
treat = df2[df2.group=='treatment']
p_treat = treat.converted.sum()/treat.shape[0]
p_treat

0.11880724790277405

In [219]:
(df2.landing_page=='new_page').sum()/df2.shape[0]

0.5000636646764286

In [220]:
p_old = df2.converted.mean()
p_new = df2.converted.mean()
n_old = df2[df2.landing_page == 'old_page'].shape[0]
n_new = df2[df2.landing_page == 'new_page'].shape[0]
print(f"p_old: {p_old}\np_new: {p_new}\nn_old: {n_old}\nn_new: {n_new}")

p_old: 0.11959667567149027
p_new: 0.11959667567149027
n_old: 145274
n_new: 145311


In [221]:
new_page_converted = np.random.binomial(n_new, p_new, 10000) 
old_page_converted = np.random.binomial(n_old, p_old, 10000)
p_diffs = new_page_converted/n_new - old_page_converted/n_old
p_diffs = np.array(p_diffs)

In [222]:
obs_diff = (df2[df2.group=='treatment'].converted.mean()) - (df2[df2.group=='control'].converted.mean())
p_val = (p_diffs > obs_diff).mean()
p_val

0.909

In [223]:
import statsmodels.api as sm

convert_old = df2[df2.landing_page=='old_page'].converted.sum()
convert_new = df2[df2.landing_page=='new_page'].converted.sum()
n_old = df2[df2.landing_page=='old_page'].shape[0]
n_new = df2[df2.landing_page=='new_page'].shape[0]

In [203]:
test_stat, p_value = sm.stats.proportions_ztest(np.array([convert_old, convert_new]), np.array([n_old, n_new]), alternative='smaller')
print(f"z-score: {test_stat}\np-value: {p_value}")

z-score: 1.3116075339133115
p-value: 0.905173705140591


In [224]:
p_new = df2.revenue.mean()
p_new

101.98272450401775

In [225]:
p_old = df2.revenue.mean()
p_old

101.98272450401775

In [166]:
df = df.groupby('group')
df.describe()

user_id                               ... revenue              
              count           mean           std  ...     50%    75%    max
group                                             ...                      
control    147202.0  788123.098035  91278.896888  ...     0.0  250.0  250.0
treatment  147276.0  787825.226283  91142.800641  ...   100.0  250.0  250.0

[2 rows x 24 columns]

In [167]:
p_all=df['revenue'].mean()
print("revenue по всем посетителям: {} %".format(p_all*100))

revenue по всем посетителям: group
control       8743.189631
treatment    11649.623836
Name: revenue, dtype: float64 %


In [226]:
n_new = df2.query('group=="treatment"').shape[0]
n_new

145311

In [227]:
n_old = df2.query('group=="control"').shape[0]
n_old

145274